In [1]:
import pandas as pd

data_train = pd.read_csv("tap_fun_train.csv")
#data_test = pd.read_csv("tap_fun_test.csv")
data_train.shape

(2288007, 109)

In [2]:
def make_lable(X):
    X['lable'] = 0
    X['lable'] = X.apply(lambda x: 1 if(x['pay_price']<x['prediction_pay_price'] and x['pay_price']!=0) else x['lable'],axis=1)
    print 'lable1'
    X['lable'] = X.apply(lambda x: 2 if(x['pay_price']==x['prediction_pay_price'] and x['pay_price']!=0) else x['lable'],axis=1)
    print 'lable2'
    X['lable'] = X.apply(lambda x: 3 if(x['pay_price']<x['prediction_pay_price'] and x['pay_price']==0) else x['lable'],axis=1)
    print 'lable3'
    X['lable'] = X.apply(lambda x: 4 if(x['pay_price']==x['prediction_pay_price'] and x['pay_price']==0) else x['lable'],axis=1)
    print 'lable4'
    return X

In [ ]:
data_train = make_lable(data_train)
data_train.shape

In [2]:
lable1 = data_train[data_train['lable']==1]
print lable1.shape
lable2 = data_train[data_train['lable']==2]
print lable2.shape
lable3 = data_train[data_train['lable']==3]
print lable3.shape
lable4 = data_train[data_train['lable']==4]
print lable4.shape

(11309, 110)
(30130, 110)
(4549, 110)
(2242019, 110)


In [3]:
lable1.to_csv('lable1.csv',index=None)
lable2.to_csv('lable2.csv',index=None)
lable3.to_csv('lable3.csv',index=None)
lable4.to_csv('lable4.csv',index=None)

In [1]:
import pandas as pd

data_train = pd.read_csv("tap_fun_train_lable.csv")
#data_test = pd.read_csv("tap_fun_test.csv")
data_train.shape

(2288007, 110)

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import RidgeClassifierCV,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.neighbors import RadiusNeighborsClassifier,NearestCentroid,KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation,LabelSpreading
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis,LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import gc
import warnings
warnings.filterwarnings('ignore')

In [5]:
class add_feature(BaseEstimator, TransformerMixin):
    def __init__(self,additional=1):
        self.additional = additional
    
    def fit(self,X,y=None):
        return self
        
    def transform(self,X):    
#        data["wkd"] = all_data.register_time.dt.weekday
        dt1=pd.to_datetime(X["register_time"])
        X["register_time"] = dt1
        X["wkd"] = dt1.dt.weekday
        X["day"] = dt1.dt.day
        X["hour"] = dt1.dt.hour
        X["day_in_year"] = dt1.dt.dayofyear
        X["days_spfest"] = X["register_time"].map(lambda x: (x - pd.to_datetime("2018-02-16")).days)
#         data =  data.merge(wkd_hour_df,on=["wkd","hour"],how = "left")
#         data["day"] = data.register_time.dt.day
#         data["month"] = data.register_time.dt.month
#         data["day_in_year"] = data.register_time.dt.dayofyear
#         data["days_spfest"] = data["register_time"].map(lambda x: (x - pd.to_datetime("2018-02-16")).days)

        X = X.drop(["register_time"],axis=1)
        if "prediction_pay_price" in X.columns:
            X = X.drop("prediction_pay_price",axis=1)
        X["price_per"] = (X["pay_price"]+0.1)/(X["pay_count"]+0.1)

        X["pvp_win_rate"] = (X.pvp_win_count + 0.1)/(X.pvp_lanch_count+0.1)
        X["pve_win_rate"] = (X.pve_win_count+0.1)/(X.pve_lanch_count+0.1)
        X["pvp_rate"] = (X.pvp_lanch_count+0.1)/(X.pve_lanch_count+0.1)
        X["pvp_vs_pve_win"] = (X.pvp_win_count+0.1)/(X.pve_win_count+0.1)
        
        X['wood'] = X['wood_add_value'] - X['wood_reduce_value'] 
        X['stone'] = X['stone_add_value'] - X['stone_reduce_value'] 
        X['ivory'] = X['ivory_add_value']  - X['ivory_reduce_value'] 
        X['meat'] =  X['meat_add_value'] - X['meat_reduce_value']
        X['magic'] =  X['magic_add_value'] - X['magic_reduce_value']
        X['infantry'] = X['infantry_add_value'] - X['infantry_reduce_value'] 
        X['cavalry'] = X['cavalry_add_value'] - X['cavalry_reduce_value'] 
        X['shaman'] = X['shaman_add_value'] - X['shaman_reduce_value'] 
        X['wound_infantry'] = X['wound_infantry_add_value'] - X['wound_infantry_reduce_value'] 
        X['wound_cavalry'] =  X['wound_cavalry_add_value'] - X['wound_cavalry_reduce_value']
        X['wound_shaman'] = X['wound_shaman_add_value'] - X['wound_shaman_reduce_value'] 
        X['general_acceleration'] = X['general_acceleration_reduce_value'] - X['general_acceleration_add_value'] 
        X['building_acceleration'] = X['building_acceleration_add_value'] - X['building_acceleration_reduce_value']
        X['reaserch_acceleration'] = X['reaserch_acceleration_add_value'] -  X['reaserch_acceleration_reduce_value']
        X['training_acceleration'] =  X['training_acceleration_add_value'] - X['training_acceleration_reduce_value']
        X['treatment_acceleration'] =  X['treatment_acceleraion_add_value'] - X['treatment_acceleration_reduce_value'] 
        
#         X = X.merge(hour_mean,on="hour",how = "left")
#         X = X.merge(wkd_mean,on="wkd",how="left")
#         X = X.merge(dayinyear_mean,how="left",on="day_in_year")
#         X = X.merge(dayinyear_hr_mean,how="left",on=["day_in_year","hour"])   
        X["avg_pay_price"] = X.pay_price/(X.avg_online_minutes+0.01)
        X["avg_pay_count"] = X.pay_count/(X.avg_online_minutes+0.01)
        X.loc[X.avg_online_minutes == 0,"avg_pay_price"] = 0
        X.loc[X.avg_online_minutes == 0,"avg_pay_count"] = 0
        X["build_remain"] = X.building_acceleration_add_value - X.building_acceleration_reduce_value
        X["build_use_rate"] = (X.building_acceleration_reduce_value + 0.1)/(X.building_acceleration_add_value + 0.1)
        X["treat_remain"] = X.treatment_acceleraion_add_value - X.treatment_acceleration_reduce_value
        X["treat_use_rate"] = (X.treatment_acceleration_reduce_value+0.1)/(X.treatment_acceleraion_add_value+0.1)
        X["res_remain"] = X.reaserch_acceleration_add_value - X.reaserch_acceleration_reduce_value
        X["res_use_rate"] = (X.reaserch_acceleration_reduce_value+ 0.1)/(X.reaserch_acceleration_add_value+ 0.1)
        #X.loc[X.avg_online_minutes == 0,"avg_acce_add"] = 0
        bd_cols_lv = [col for col in X.columns if col.startswith("bd")]
        sr_cols_lv = [col for col in X.columns if col.startswith("sr")]
        X["pve_vs_price"] = X["pve_win_count"]/(X["pay_price"]+0.1)
        X["pvp_vs_price"] = X["pvp_win_count"]/(X["pay_price"]+0.1)
        X["total_bd_lv"] = X[bd_cols_lv].sum(axis=1)
        X["total_sr_lv"] = X[sr_cols_lv].sum(axis=1)
        X["total_lv"] = X["total_bd_lv"] + X["total_sr_lv"]
        X["sr_prod_lv"] = X[['sr_rss_a_prod_levell','sr_rss_b_prod_level','sr_rss_c_prod_level','sr_rss_d_prod_level']].sum(axis=1)
        X["ivory_use_rate"] = (X["ivory_reduce_value"] + 0.1)/(X["ivory_add_value"] + 0.1)
        X["ivory_remain"] = X["ivory_add_value"] - X["ivory_reduce_value"]
        gc.collect()
        return X

In [6]:
pipe = Pipeline([
    ('add_feature', add_feature(additional=2))
    ])

data_train = pipe.fit_transform(data_train)

In [3]:
data_train.drop(['user_id'],axis=1, inplace=True)
if "prediction_pay_price" in data_train.columns:
    data_train = data_train.drop("prediction_pay_price",axis=1)
    dt1=pd.to_datetime(data_train["register_time"])
    data_train["register_time"] = dt1.dt.dayofyear
data_train.shape

(2288007, 108)

In [9]:
data_train['lable_2'] = data_train['lable']
del data_train['lable']
data_train.rename(columns={'lable_2':'lable'}, inplace = True)

In [4]:
x, y = data_train.iloc[:, 0:len(data_train.columns)-1], data_train.iloc[:,len(data_train.columns)-1:]
x.shape,y.columns

((2288007, 107), Index([u'lable'], dtype='object'))

In [11]:
x = MinMaxScaler().fit_transform(x)

In [12]:
pca = PCA(n_components=100)

In [13]:
x=pca.fit_transform(x)
print x.shape

(91976L, 100L)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5)

In [7]:
# lr = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), cv=3)
# lr.fit(x_train, y_train)
# print u'参数alpha=%.2f' % lr.alpha_
# y_train_pred = lr.predict(x_train)
# y_test_pred = lr.predict(x_test)
# print u'Logistic回归训练集准确率：', accuracy_score(y_train, y_train_pred)
# print u'Logistic回归测试集准确率：', accuracy_score(y_test, y_test_pred)

rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_split=5, oob_score=True)
rf.fit(x_train, y_train)
print u'OOB Score=%.5f' % rf.oob_score_
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)
print u'随机森林训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'随机森林测试集准确率：', accuracy_score(y_test, y_test_pred)


In [13]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
gb.fit(x_train, y_train)
y_train_pred = gb.predict(x_train)
y_test_pred = gb.predict(x_test)
print u'GBDT训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'GBDT测试集准确率：', accuracy_score(y_test, y_test_pred)

GBDT训练集准确率： 0.9959038568954802
GBDT测试集准确率： 0.9944414530019126


In [8]:
def lightGBM(train_x,train_y,validation_x,validation_y):
    clf = lgb.LGBMClassifier(num_leaves= 127, 
                             max_depth=10,
                             num_class=4,
                             silent=True,
                             metric='auc',
                             n_jobs=8, 
                             n_estimators=1000,
                             colsample_bytree=0.9,
                             subsample=0.9,
                             learning_rate=0.01
                            )
#     clf = lgb.LGBMClassifier(boosting_type='gbdt',
#                              num_class=4, 
#                              num_leaves=55, 
#                              reg_alpha=0.0, 
#                              reg_lambda=1,
#                              max_depth=12, 
#                              n_estimators=1000, 
#                              objective='multiclass',
#                              subsample=0.8, 
#                              colsample_bytree=0.8, 
#                              subsample_freq=1,
#                              learning_rate=0.01, 
#                              min_child_weight=1, 
#                              random_state=20, 
#                              n_jobs=4
#                             )
    
    # clf.fit(train_x, train_y, **fit_params)
    clf.fit(train_x,
    train_y,
    early_stopping_rounds=100,
    eval_metric= 'auc', 
            eval_set = [(validation_x,validation_y)],
            eval_names = ['valid'],
            verbose= 10
    )
         
    return clf

clf = lightGBM(x_train,y_train, x_test, y_test)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
print u'lgb训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'lgb测试集准确率：', accuracy_score(y_test, y_test_pred)


Training until validation scores don't improve for 100 rounds.
[10]	valid's auc: 0.00205102
[20]	valid's auc: 0.00202161
[30]	valid's auc: 0.00202037
[40]	valid's auc: 0.00201462
[50]	valid's auc: 0.00201275
[60]	valid's auc: 0.00200585
[70]	valid's auc: 0.00199781
[80]	valid's auc: 0.00199582
[90]	valid's auc: 0.00198723
[100]	valid's auc: 0.0019806
Early stopping, best iteration is:
[1]	valid's auc: 0.00233898
lgb训练集准确率： 0.9950673206276557
lgb测试集准确率： 0.994323446421516


In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
dtc.fit(x_train, y_train)
y_train_pred = dtc.predict(x_train)
y_test_pred = dtc.predict(x_test)
print u'dtc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'dtc测试集准确率：', accuracy_score(y_test, y_test_pred)                           
                          

etc = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
etc.fit(x_train, y_train)
y_train_pred = etc.predict(x_train)
y_test_pred = etc.predict(x_test)
print u'etc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'etc测试集准确率：', accuracy_score(y_test, y_test_pred)

dtc训练集准确率： 0.9998706297098872
dtc测试集准确率： 0.990913493309464


F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


etc训练集准确率： 0.9998706297098872
etc测试集准确率： 0.9934169810595068


In [7]:
bnb = BernoulliNB()
bnb.fit(x_train, y_train)
y_train_pred = bnb.predict(x_train)
y_test_pred = bnb.predict(x_test)
print u'bnb训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'bnb测试集准确率：', accuracy_score(y_test, y_test_pred)  

F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


bn训练集准确率： 0.9029950096284712
bn测试集准确率： 0.9028281369645561


In [10]:
bayes = GaussianNB() 
bayes.fit(x_train, y_train)
y_train_pred = bayes.predict(x_train)
y_test_pred = bayes.predict(x_test)
print u'bayes训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'bayes测试集准确率：', accuracy_score(y_test, y_test_pred)  

gn训练集准确率： 0.9661390748101185
gn测试集准确率： 0.9660761675658477


In [6]:
lp = LabelPropagation()#MemoryError
lp.fit(x_train, y_train)
y_train_pred = lp.predict(x_train)
y_test_pred = lp.predict(x_test)
print u'lp训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'lp测试集准确率：', accuracy_score(y_test, y_test_pred)  

In [23]:
RidgeClassifier()
rc.fit(x_train, y_train)
y_train_pred = rc.predict(x_train)
y_test_pred = rc.predict(x_test)
print u'rc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'rc测试集准确率：', accuracy_score(y_test, y_test_pred)  

rc训练集准确率： 0.9848802844048485
rc测试集准确率： 0.9847867664798374


In [ ]:
rnc = RadiusNeighborsClassifier() #MemoryError
rnc.fit(x_train, y_train)
y_train_pred = rnc.predict(x_train)
y_test_pred = rnc.predict(x_test)
print u'rnc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'rnc测试集准确率：', accuracy_score(y_test, y_test_pred) 

F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [7]:
qda = QuadraticDiscriminantAnalysis() 
qda.fit(x_train, y_train)
y_train_pred = qda.predict(x_train)
y_test_pred = qda.predict(x_test)
print u'qda训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'qda测试集准确率：', accuracy_score(y_test, y_test_pred)

F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


qda训练集准确率： 0.9578908446918408
qda测试集准确率： 0.9578060915870924


In [ ]:
nc = NearestCentroid() 
nc.fit(x_train, y_train)
y_train_pred = nc.predict(x_train)
y_test_pred = nc.predict(x_test)
print u'nc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'cn测试集准确率：', accuracy_score(y_test, y_test_pred) 

In [ ]:
mlp = sklearn.neural_network.MLPClassifier() 
mlp.fit(x_train, y_train)
y_train_pred = mlp.predict(x_train)
y_test_pred = mlp.predict(x_test)
print u'mlp训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'mlp测试集准确率：', accuracy_score(y_test, y_test_pred)  

In [12]:
lda = LinearDiscriminantAnalysis() 
lda.fit(x_train, y_train)
y_train_pred = lda.predict(x_train)
y_test_pred = lda.predict(x_test)
print u'lda训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'lda测试集准确率：', accuracy_score(y_test, y_test_pred) 

F:\zhangzhenhua\anaconda2.7\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


lda训练集准确率： 0.9757063574133984
lda测试集准确率： 0.9757090010174789


In [7]:
ls = LabelSpreading() #MemoryError 
ls.fit(x_train, y_train)
y_train_pred = ls.predict(x_train)
y_test_pred = ls.predict(x_test)
print u'ls训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'ls测试集准确率：', accuracy_score(y_test, y_test_pred) 

In [9]:
knc = KNeighborsClassifier() 
knc.fit(x_train, y_train)
y_train_pred = knc.predict(x_train)
y_test_pred = knc.predict(x_test)
print u'knc训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'knc测试集准确率：', accuracy_score(y_test, y_test_pred) 

F:\zhangzhenhua\anaconda2.7\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


knc训练集准确率： 0.9896774746220071
knc测试集准确率： 0.9869222485236065


In [6]:
bagging = BaggingClassifier()
bagging.fit(x_train, y_train)
y_train_pred = bagging.predict(x_train)
y_test_pred = bagging.predict(x_test)
print u'bagging训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'bagging测试集准确率：', accuracy_score(y_test, y_test_pred) 

bagging训练集准确率： 0.9995585675911689
bagging测试集准确率： 0.9939257205394387


In [6]:
Ada = AdaBoostClassifier()
Ada.fit(x_train, y_train)
y_train_pred = Ada.predict(x_train)
y_test_pred = Ada.predict(x_test)
print u'Ada训练集准确率：', accuracy_score(y_train, y_train_pred)
print u'Ada测试集准确率：', accuracy_score(y_test, y_test_pred) 

Ada训练集准确率： 0.9931040390628346
Ada测试集准确率： 0.9930341152653313


In [7]:
all_test=pd.read_csv('tap_fun_test.csv')

In [8]:
fit_data = all_test.iloc[:,1:108]
dt1=pd.to_datetime(fit_data["register_time"])
fit_data["register_time"] = dt1.dt.dayofyear

In [9]:
y_pred = bagging.predict(fit_data.values)
y_pred = pd.DataFrame(y_pred)
all_test['lable']=y_pred

In [10]:
print all_test.columns
all_test.to_csv('tap_fun_test_lable_bagging.csv',index=None)

Index([u'user_id', u'register_time', u'wood_add_value', u'wood_reduce_value',
       u'stone_add_value', u'stone_reduce_value', u'ivory_add_value',
       u'ivory_reduce_value', u'meat_add_value', u'meat_reduce_value',
       ...
       u'pvp_battle_count', u'pvp_lanch_count', u'pvp_win_count',
       u'pve_battle_count', u'pve_lanch_count', u'pve_win_count',
       u'avg_online_minutes', u'pay_price', u'pay_count', u'lable'],
      dtype='object', length=109)


In [11]:
lable1 = all_test[all_test['lable']==1]
print lable1.shape
lable2 = all_test[all_test['lable']==2]
print lable2.shape
lable3 = all_test[all_test['lable']==3]
print lable3.shape
lable4 = all_test[all_test['lable']==4]
print lable4.shape

(5313, 109)
(14236, 109)
(194, 109)
(809191, 109)


In [30]:
all_test['prediction_pay_price'] = 0
all_test['prediction_pay_price'] = all_test.apply(lambda x: 4.26*x['pay_price'] if(x['lable'] == 1 ) else x['prediction_pay_price'],axis=1)
print 'lable1'
all_test['prediction_pay_price'] = all_test.apply(lambda x: x['pay_price'] if(x['lable'] == 2 ) else x['prediction_pay_price'],axis=1)
print 'lable2'
all_test['prediction_pay_price'] = all_test.apply(lambda x: x['pay_price'] if(x['lable'] == 3 ) else x['prediction_pay_price'],axis=1)
print 'lable2'

In [ ]:
sklearn.naive_bayes.BernoulliNB sklearn.tree.DecisionTreeClassifier sklearn.tree.ExtraTreeClassifier sklearn.ensemble.ExtraTreesClassifier sklearn.naive_bayes.GaussianNB sklearn.neighbors.KNeighborsClassifier sklearn.semi_supervised.LabelPropagation sklearn.semi_supervised.LabelSpreading sklearn.discriminant_analysis.LinearDiscriminantAnalysis sklearn.svm.LinearSVC (setting multi_class=”crammer_singer”) sklearn.linear_model.LogisticRegression (setting multi_class=”multinomial”) sklearn.linear_model.LogisticRegressionCV (setting multi_class=”multinomial”) sklearn.neural_network.MLPClassifier sklearn.neighbors.NearestCentroid sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis sklearn.neighbors.RadiusNeighborsClassifier sklearn.ensemble.RandomForestClassifier sklearn.linear_model.RidgeClassifier sklearn.linear_model.RidgeClassifierCV